In [1]:
import pandas as pd
import csv
from datetime import datetime, timedelta

In [2]:
'''
Instructions:
1. Put input CSVs inside the directory specified in the input_folder variable
2. Change the date_subject to the date today. Follow the format YYYYMMDD
3. Run the notebook
4. Get the output from the output folder
5. Upload the files to https://drive.google.com/drive/folders/1QelvFeULpkytJxkWF8XnBOpK7sV6Pk80

'''

date_subject = 20201206 
date_today = 20201206
output_date = datetime.now() - timedelta(date_today-date_subject)
aa_date = output_date - timedelta(2)

input_folder = 'Sources/'
lookup_folder = 'Lookup Files/'
output_folder = 'For Validation/'


In [3]:
def movecol(df, cols_to_move=[], ref_col='', place='After'):
    
    cols = df.columns.tolist()
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]
    
    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]
    
    return(df[seg1 + seg2 + seg3])

In [5]:
# Lookup files for activity window-date lookup

d7 = pd.read_csv(lookup_folder+"7day.csv")
d30 = pd.read_csv(lookup_folder+"30day.csv")
d90 = pd.read_csv(lookup_folder+"90day.csv")
d365 = pd.read_csv(lookup_folder+"365day.csv")

from functools import reduce
window_date_lookup = reduce(lambda left,right: pd.merge(left,right,on='date'), [d7, d30, d90, d365])
window_date_lookup.columns = ['date', '7d', '30d', '90d', '365d']
window_date_lookup['date'] = pd.to_datetime(window_date_lookup['date']) 
window_date_lookup.set_index('date', inplace=True)
window_date_lookup

,7d,30d,90d,365d
date,,,,
2019-05-27,1,898,1795,2692
2019-05-28,2,899,1796,2693
2019-05-29,3,900,1797,2694
2019-05-30,4,901,1798,2695
2019-05-31,5,902,1799,2696
...,...,...,...,...
2019-04-03,8194,8201,8208,8215
2019-04-04,8195,8202,8209,8216
2019-04-05,8196,8203,8210,8217


In [8]:
country_lookup = pd.read_csv(lookup_folder+"countries_mapped_iso3_metadata.csv")
country_lookup.set_index('adobe_analytics_country', inplace=True)
country_lookup = country_lookup[['iso3']]
country_lookup

,iso3
adobe_analytics_country,
andorra,AND
austria,AUT
azerbaijan,AZE
belize,BLZ
cayman islands,CYM
...,...
russian federation,RUS
st. helena,SHN
syrian arab republic,SYR


In [9]:
# Step 6

# Change input and output directories
input_filename_6 = input_folder+'GDS_WIG3_-_Mobilizing_Digital_Missionaries - '+ str(date_subject)+'.csv'
output_filename_6 = output_folder+output_date.strftime('%Y%m%d') + '_step6_29.csv'  # default to today

input = open(input_filename_6, 'rb')

pre_csv = list(input.readlines())
rows_to_remove = [r - 1 for r in [13,11,10,9,8,7,6,5,4,3,2,1]]

for r in rows_to_remove:
    pre_csv.pop(r)


pre_csv=[x.rstrip().decode('utf-8').split(",") for x in pre_csv]
    
df = pd.DataFrame(pre_csv[1:pre_csv.index([''])],columns=pre_csv[0])


df.columns = ['country', '365d', '90d', '30d', '7d']

df.at[0, 'country'] = 'Global'
df['dateValue'] = aa_date.strftime('%Y-%m-%d')  # default to yesterday, if report is generated today
df['product'] = 'godtools'

df = movecol(df, 
             cols_to_move=['product','country', '7d', '30d', '90d', '365d'], 
             ref_col='dateValue',
             place='After')


df = df[~df.country.isin(['', None])]
df = df.dropna(subset=['country', '7d']) 


df.columns = ['date','product','country','7d','30d','90d','365d']
df['source_id'] = 29
df = df.melt(id_vars=['date','product', 'source_id', 'country'],var_name = 'window', value_name = 'value')
df['date'] =  pd.to_datetime(df['date']) 
df.country = [str(x).lower() for x in df.country]

df['isOriginal'] = 1
df['activitywindow_date_id'] = window_date_lookup.lookup(df.date, df.window)
temp=df.join(country_lookup, on='country')
temp.country = temp.iso3
temp.drop(columns=['iso3'], inplace=True)
df = temp

final_step = df.copy()

final_step['record_id'] = ''
final_step.drop(columns=['date', 'product', 'window'], inplace=True)
final_step = movecol(final_step, 
             cols_to_move=['activitywindow_date_id', 'source_id', 'country', 'value', 'isOriginal'], 
             ref_col='record_id',
             place='After')

final_step.to_csv(output_filename_6, encoding='utf-8', header=True, index=False)
final_step




,record_id,activitywindow_date_id,source_id,country,value,isOriginal
0,,558,29,GGG,518,1
1,,558,29,IDN,97,1
2,,558,29,VEN,66,1
3,,558,29,MEX,48,1
4,,558,29,COL,40,1
...,...,...,...,...,...,...
775,,3249,29,ISL,1,1
776,,3249,29,FSM,1,1
777,,3249,29,DNK,1,1
778,,3249,29,NCL,1,1


In [7]:
# Step 6.5

input_filename_65 = input_folder+'GDS_WIG3_-_Mobilizing_Digital_Missionaries - '+ str(date_subject)+'.csv'
output_filename_65 = output_folder+output_date.strftime('%Y%m%d') + '_step6_30.csv'  # default to today

input = open(input_filename_65, 'rb')

pre_csv = list(input.readlines())
pre_csv=[x.rstrip().decode('utf-8').split(",") for x in pre_csv]
pre_csv = pre_csv[pre_csv.index(['# MissionHub Digital Missionaries']):]
rows_to_remove = [r - 1 for r in [12,11,10,9,8,7,6,5,4,3,2,1]]

for r in rows_to_remove:
    pre_csv.pop(r)


    
df = pd.DataFrame(pre_csv[0:],columns=pre_csv[0])
df.columns = ['country', '365d', '90d', '30d', '7d']
df.at[0, 'country'] = 'Global'
df['dateValue'] = aa_date.strftime('%Y-%m-%d')  # default to yesterday, if report is generated today
df['product'] = 'missionhub'
df

df = movecol(df, 
             cols_to_move=['product', 'country', '7d', '30d', '90d', '365d'], 
             ref_col='dateValue',
             place='After')

df = df[~df.country.isin(['', 'Countries', None])]
df = df.dropna(subset=['country', '7d']) 

df.to_csv(output_filename_65, encoding='utf-8', header=False, index=False)

df.columns = ['date','product','country','7d','30d','90d','365d']
df['source_id'] = 30
df = df.melt(id_vars=['date','product', 'source_id', 'country'],var_name = 'window', value_name = 'value')
df['date'] =  pd.to_datetime(df['date']) 
df.country = [str(x).lower() for x in df.country]

df['isOriginal'] = 1
df['activitywindow_date_id'] = window_date_lookup.lookup(df.date, df.window)
temp=df.join(country_lookup, on='country')
temp.country = temp.iso3
temp.drop(columns=['iso3'], inplace=True)
df = temp

final_step = df.copy()

final_step['record_id'] = ''
final_step.drop(columns=['date', 'product', 'window'], inplace=True)
final_step = movecol(final_step, 
             cols_to_move=['activitywindow_date_id', 'source_id', 'country', 'value', 'isOriginal'], 
             ref_col='record_id',
             place='After')

final_step.to_csv(output_filename_65, encoding='utf-8', header=True, index=False)
final_step

,record_id,activitywindow_date_id,source_id,country,value,isOriginal
0,,558,30,GGG,76,1
1,,558,30,USA,26,1
2,,558,30,GTM,13,1
3,,558,30,ARG,8,1
4,,558,30,MEX,4,1
...,...,...,...,...,...,...
487,,3249,30,LBN,1,1
488,,3249,30,FIN,1,1
489,,3249,30,GRC,1,1
490,,3249,30,QAT,1,1


In [10]:
# Step 8

input_filename_8 = input_folder+'Signups by country - '+ str(date_subject)+'.csv'
output_filename_8 = output_folder+output_date.strftime('%Y%m%d') + '_step8.csv'  # default to today

input = open(input_filename_8, 'rb')

pre_csv = list(input.readlines())
pre_csv=[x.rstrip().decode('utf-8').split(",") for x in pre_csv]
rows_to_remove = [r - 1 for r in [13,12,11,10,9,8,7,6,5,4,3,2,1]]

for r in rows_to_remove:
    pre_csv.pop(r)
    

df = pd.DataFrame(pre_csv[0:],columns=pre_csv[0])

df.columns = ['country', '7d', '30d', '90d', '365d']
df['product'] = 'es-email-web-signups'
df['threshold'] = 'curious'
df['type'] = 'email'
df['subtype'] = ''
df['dateValue'] = aa_date.strftime('%Y-%m-%d')  # default to 2 days ago

df = movecol(df, 
             cols_to_move=[ 'product', '7d', '30d', '90d', '365d', 'threshold', 'country', 'type', 'subtype'],
             ref_col='dateValue',
             place='After')

df = df[~df.country.isin(['', None])]
df.at[0, 'country'] = 'Global'


df.columns = ['date','product','7d','30d','90d','365d','threshold', 'country', 'email', 'nan']
df['source_id'] = 76
df['threshold'] = 1 # Curious
df = df.melt(id_vars=['date','product', 'source_id', 'country', 'email', 'threshold', 'nan'],var_name = 'window', value_name = 'value')
df['date'] =  pd.to_datetime(df['date']) 
df.country = [str(x).lower() for x in df.country]

df['isOriginal'] = 1
df['activitywindow_date_id'] = window_date_lookup.lookup(df.date, df.window)
temp=df.join(country_lookup, on='country')
temp.country = temp.iso3
temp.drop(columns=['iso3'], inplace=True)
df = temp

final_step = df.copy()

final_step['record_id'] = ''
final_step.drop(columns=['date', 'product', 'window', 'email', 'nan'], inplace=True)
final_step = movecol(final_step, 
             cols_to_move=['activitywindow_date_id', 'source_id', 'country', 'value', 'isOriginal', 'threshold'], 
             ref_col='record_id',
             place='After')
final_step.to_csv(output_filename_8, encoding='utf-8', header=True, index=False)

final_step

,record_id,activitywindow_date_id,source_id,country,value,isOriginal,threshold
0,,558,76,GGG,784,1,1
1,,558,76,BRA,523,1,1
2,,558,76,COL,4,1,1
3,,558,76,MEX,5,1,1
4,,558,76,USA,37,1,1
...,...,...,...,...,...,...,...
727,,3249,76,NU2,1,1,1
728,,3249,76,MAC,1,1,1
729,,3249,76,PSE,1,1,1
730,,3249,76,MNE,1,1,1


In [9]:
# Step 15

input_filename_15 = input_folder+'Exposures by Country - Cru Global Report Suite - '+ str(date_subject)+'.csv'
output_filename_15 = output_folder+output_date.strftime('%Y%m%d') + '_step15.csv'  # default to today


input = open(input_filename_15, 'rb')

pre_csv = list(input.readlines())
rows_to_remove = [r - 1 for r in [12,10,9,8,7,6,5,4,3,2,1]]

for r in rows_to_remove:
    pre_csv.pop(r)


pre_csv=[x.rstrip().decode('utf-8').split(",") for x in pre_csv]
    
df = pd.DataFrame(pre_csv[1:],columns=pre_csv[0])
df.columns = ['country', '7d', '30d', '90d', '365d']
df['dateValue'] = aa_date.strftime('%Y-%m-%d')  # default to 2 days ago, if report is generated today
df['source'] = 'adobe-analytics'

df = movecol(df, 
             cols_to_move=['source', 'country', '7d', '30d', '90d', '365d'], 
             ref_col='dateValue',
             place='After')

df = df[~df.country.isin(['', None])]
df.at[0, 'country'] = 'Global'


df.columns = ['date','product','country','7d','30d','90d','365d']
df['source_id'] = 50
df = df.melt(id_vars=['date','product', 'source_id', 'country'],var_name = 'window', value_name = 'value')
df['date'] =  pd.to_datetime(df['date']) 
df.country = [str(x).lower() for x in df.country]

df['isOriginal'] = 1
df['activitywindow_date_id'] = window_date_lookup.lookup(df.date, df.window)
temp=df.join(country_lookup, on='country')
temp.country = temp.iso3
temp.drop(columns=['iso3'], inplace=True)
df = temp

final_step = df.copy()

final_step['record_id'] = ''
final_step.drop(columns=['date', 'product', 'window'], inplace=True)
final_step = movecol(final_step, 
             cols_to_move=['activitywindow_date_id', 'source_id', 'country', 'value', 'isOriginal'], 
             ref_col='record_id',
             place='After')
final_step.to_csv(output_filename_15, encoding='utf-8', header=True, index=False)

final_step

        

,record_id,activitywindow_date_id,source_id,country,value,isOriginal
0,,558,50,GGG,722079,1
1,,558,50,USA,206558,1
2,,558,50,BRA,58384,1
3,,558,50,MEX,6928,1
4,,558,50,TUR,36382,1
...,...,...,...,...,...,...
959,,3249,50,TKL,7,1
960,,3249,50,PRK,4,1
961,,3249,50,SHN,2,1
962,,3249,50,BLM,1,1


In [10]:
# Source 92

input_filename_6 = input_folder+'Decision_Tracking_for_GDS_Dashboard - '+ str(date_subject)+'.csv'
output_filename_6 = output_folder+output_date.strftime('%Y%m%d') + '_source_92.csv'  # default to today

input = open(input_filename_6, 'rb')

pre_csv = list(input.readlines())
rows_to_remove = [r - 1 for r in [12,11,10,9,8,7,6,5,4,3,2,1]]

for r in rows_to_remove:
    pre_csv.pop(r)


pre_csv=[x.rstrip().decode('utf-8').split(",") for x in pre_csv]
    
df = pd.DataFrame(pre_csv[1:pre_csv.index([''])],columns=pre_csv[0])


df.columns = ['country', '365d', '90d', '30d', '7d']

df.at[0, 'country'] = 'Global'
df['dateValue'] = aa_date.strftime('%Y-%m-%d')  # default to yesterday, if report is generated today
df['product'] = 'godtools'

df = movecol(df, 
             cols_to_move=['product','country', '7d', '30d', '90d', '365d'], 
             ref_col='dateValue',
             place='After')


df = df[~df.country.isin(['', None])]
df = df.dropna(subset=['country', '7d']) 

df.columns = ['date','product','country','7d','30d','90d','365d']
df['source_id'] = 92
df = df.melt(id_vars=['date','product', 'source_id', 'country'],var_name = 'window', value_name = 'value')
df['date'] =  pd.to_datetime(df['date']) 
df.country = [str(x).lower() for x in df.country]

df['isOriginal'] = 1
df['activitywindow_date_id'] = window_date_lookup.lookup(df.date, df.window)
temp=df.join(country_lookup, on='country')
temp.country = temp.iso3
temp.drop(columns=['iso3'], inplace=True)
df = temp

final_step = df.copy()

final_step['record_id'] = ''
final_step.drop(columns=['date', 'product', 'window'], inplace=True)
final_step = movecol(final_step, 
             cols_to_move=['activitywindow_date_id', 'source_id', 'country', 'value', 'isOriginal'], 
             ref_col='record_id',
             place='After')

final_step.to_csv(output_filename_6, encoding='utf-8', header=True, index=False)
final_step




,record_id,activitywindow_date_id,source_id,country,value,isOriginal
0,,558,92,GGG,450,1
1,,558,92,NER,0,1
2,,558,92,ARM,0,1
3,,558,92,TJK,0,1
4,,558,92,LBY,0,1
...,...,...,...,...,...,...
671,,3249,92,USA,2763,1
672,,3249,92,COL,1226,1
673,,3249,92,MEX,1500,1
674,,3249,92,VEN,1387,1


In [11]:
# Source 93

input_filename_6 = input_folder+'Decision_Tracking_for_GDS_Dashboard - '+ str(date_subject)+'.csv'
output_filename_6 = output_folder+output_date.strftime('%Y%m%d') + '_source_93.csv'  # default to today

input = open(input_filename_6, 'rb')

pre_csv = list(input.readlines())
pre_csv=[x.rstrip().decode('utf-8').split(",") for x in pre_csv]
pre_csv = pre_csv[pre_csv.index(['# source 93- LMI - win (non-godtools)']):pre_csv.index(['"# source 94 - Decisions for Christ (ES- pageview method) "'])]
rows_to_remove = [r - 1 for r in [4,3,2,1]]

for r in rows_to_remove:
    pre_csv.pop(r)

df = pd.DataFrame(pre_csv[1:pre_csv.index([''])],columns=pre_csv[0])

df.columns = ['country', '90d', '30d', '7d']

df.at[0, 'country'] = 'Global'
df['dateValue'] = aa_date.strftime('%Y-%m-%d')  # default to yesterday, if report is generated today
df['product'] = 'godtools'

df = movecol(df, 
             cols_to_move=['product','country', '7d', '30d', '90d'], 
             ref_col='dateValue',
             place='After')


df = df[~df.country.isin(['', None])]
df = df.dropna(subset=['country', '7d']) 

#df.to_csv(output_filename_6, encoding='utf-8', header=False, index=False)



df.columns = ['date','product','country','7d','30d','90d']
df['source_id'] = 93
df = df.melt(id_vars=['date','product', 'source_id', 'country'],var_name = 'window', value_name = 'value')
df['date'] =  pd.to_datetime(df['date']) 
df.country = [str(x).lower() for x in df.country]

df['isOriginal'] = 1
df['activitywindow_date_id'] = window_date_lookup.lookup(df.date, df.window)
temp=df.join(country_lookup, on='country')
temp.country = temp.iso3
temp.drop(columns=['iso3'], inplace=True)
df = temp

final_step = df.copy()

final_step['record_id'] = ''
final_step.drop(columns=['date', 'product', 'window'], inplace=True)
final_step = movecol(final_step, 
             cols_to_move=['activitywindow_date_id', 'source_id', 'country', 'value', 'isOriginal'], 
             ref_col='record_id',
             place='After')

final_step.to_csv(output_filename_6, encoding='utf-8', header=True, index=False)
final_step


,record_id,activitywindow_date_id,source_id,country,value,isOriginal
0,,558,93,GGG,1966,1
1,,558,93,TUR,346,1
2,,558,93,IND,151,1
3,,558,93,RUS,309,1
4,,558,93,POL,137,1
...,...,...,...,...,...,...
445,,2352,93,ABW,1,1
446,,2352,93,BHS,1,1
447,,2352,93,ZWE,1,1
448,,2352,93,ARE,1,1


In [12]:
# Source 94

input_filename_6 = input_folder+'Decision_Tracking_for_GDS_Dashboard - '+ str(date_subject)+'.csv'
output_filename_6 = output_folder+output_date.strftime('%Y%m%d') + '_source_94.csv'  # default to today

input = open(input_filename_6, 'rb')

pre_csv = list(input.readlines())
pre_csv=[x.rstrip().decode('utf-8').split(",") for x in pre_csv]
pre_csv = pre_csv[pre_csv.index(['"# source 94 - Decisions for Christ (ES- pageview method) "']):pre_csv.index(['# source 95 - Decisions for Christ (godtools) app-method'
])]
rows_to_remove = [r - 1 for r in [3,2,1]]

for r in rows_to_remove:
    pre_csv.pop(r)

    
df = pd.DataFrame(pre_csv[1:pre_csv.index([''])],columns=pre_csv[0])


df.columns = ['country', '365d', '90d', '30d', '7d']

df.at[0, 'country'] = 'Global'
df['dateValue'] = aa_date.strftime('%Y-%m-%d')  # default to yesterday, if report is generated today
df['product'] = 'godtools'

df = movecol(df, 
             cols_to_move=['product','country', '7d', '30d', '90d', '365d'], 
             ref_col='dateValue',
             place='After')


df = df[~df.country.isin(['', None])]
df = df.dropna(subset=['country', '7d']) 

#df.to_csv(output_filename_6, encoding='utf-8', header=False, index=False)



df.columns = ['date','product','country','7d','30d','90d','365d']
df['source_id'] = 94
df = df.melt(id_vars=['date','product', 'source_id', 'country'],var_name = 'window', value_name = 'value')
df['date'] =  pd.to_datetime(df['date']) 
df.country = [str(x).lower() for x in df.country]

df['isOriginal'] = 1
df['activitywindow_date_id'] = window_date_lookup.lookup(df.date, df.window)
temp=df.join(country_lookup, on='country')
temp.country = temp.iso3
temp.drop(columns=['iso3'], inplace=True)
df = temp

final_step = df.copy()

final_step['record_id'] = ''
final_step.drop(columns=['date', 'product', 'window'], inplace=True)
final_step = movecol(final_step, 
             cols_to_move=['activitywindow_date_id', 'source_id', 'country', 'value', 'isOriginal'], 
             ref_col='record_id',
             place='After')

final_step.to_csv(output_filename_6, encoding='utf-8', header=True, index=False)
final_step


,record_id,activitywindow_date_id,source_id,country,value,isOriginal
0,,558,94,GGG,4637,1
1,,558,94,NGA,1694,1
2,,558,94,TUR,446,1
3,,558,94,MEX,8,1
4,,558,94,RUS,360,1
...,...,...,...,...,...,...
811,,3249,94,DMA,1,1
812,,3249,94,CYM,1,1
813,,3249,94,BMU,1,1
814,,3249,94,SXM,1,1


In [13]:
# Source 95

input_filename_6 = input_folder+'Decision_Tracking_for_GDS_Dashboard - '+ str(date_subject)+'.csv'
output_filename_6 = output_folder+output_date.strftime('%Y%m%d') + '_source_95.csv'  # default to today

input = open(input_filename_6, 'rb')

pre_csv = list(input.readlines())
pre_csv=[x.rstrip().decode('utf-8').split(",") for x in pre_csv]
pre_csv = pre_csv[pre_csv.index(['# source 95 - Decisions for Christ (godtools) app-method']):pre_csv.index(['# 30-Day Window'
])]
rows_to_remove = [r - 1 for r in [3,2,1]]

for r in rows_to_remove:
    pre_csv.pop(r)

    
df = pd.DataFrame(pre_csv[1:pre_csv.index([''])],columns=pre_csv[0])


df.columns = ['country', '365d', '90d', '30d', '7d']

df.at[0, 'country'] = 'Global'
df['dateValue'] = aa_date.strftime('%Y-%m-%d')  # default to yesterday, if report is generated today
df['product'] = 'godtools'

df = movecol(df, 
             cols_to_move=['product','country', '7d', '30d', '90d', '365d'], 
             ref_col='dateValue',
             place='After')


df = df[~df.country.isin(['', None])]
df = df.dropna(subset=['country', '7d']) 

#df.to_csv(output_filename_6, encoding='utf-8', header=False, index=False)



df.columns = ['date','product','country','7d','30d','90d','365d']
df['source_id'] = 95
df = df.melt(id_vars=['date','product', 'source_id', 'country'],var_name = 'window', value_name = 'value')
df['date'] =  pd.to_datetime(df['date']) 
df.country = [str(x).lower() for x in df.country]

df['isOriginal'] = 1
df['activitywindow_date_id'] = window_date_lookup.lookup(df.date, df.window)
temp=df.join(country_lookup, on='country')
temp.country = temp.iso3
temp.drop(columns=['iso3'], inplace=True)
df = temp

final_step = df.copy()

final_step['record_id'] = ''
final_step.drop(columns=['date', 'product', 'window'], inplace=True)
final_step = movecol(final_step, 
             cols_to_move=['activitywindow_date_id', 'source_id', 'country', 'value', 'isOriginal'], 
             ref_col='record_id',
             place='After')

final_step.to_csv(output_filename_6, encoding='utf-8', header=True, index=False)
final_step



,record_id,activitywindow_date_id,source_id,country,value,isOriginal
0,,558,95,GGG,450,1
1,,558,95,MEX,55,1
2,,558,95,VEN,95,1
3,,558,95,IDN,81,1
4,,558,95,USA,32,1
...,...,...,...,...,...,...
671,,3249,95,IRN,1,1
672,,3249,95,TJK,1,1
673,,3249,95,LBY,1,1
674,,3249,95,GNB,1,1
